## Image Processing Using Keras and Tensorflow - Chris Kim

This notebook was adapted from github user [nirmalyaghosh](https://github.com/nirmalyaghosh/deep-learning-vm), and user [fchollet](https://github.com/fchollet/keras/issues/3109). Lastly, most information was adapted from a great website called [Machine Learning Mastery](http://machinelearningmastery.com/).

The code below is my first attempt to use image processing using the tools made available in Python, namely `keras` and `TensorFlow` as the backend. This exploration was inspired by [doyleax](https://doyleax.github.io/Portfolio/capstone.html) and her Capstone project. Side profile images of 1780 pairs of shoes are transformed to be evaluated in a neural network to try to predict whether a shoe is classified as rare or not (determined by a metric composed of retail price and average deadstock price).

All images are taken from [StockX](www.stockx.com).

In [25]:
import numpy as np
import pickle
import os

np.random.seed(7)
%matplotlib inline

In [26]:
with open('../../../keras/deep-learning-vm/notebooks/shoenumbers.pickle', 'rb') as handle:
    X = pickle.load(handle, encoding='bytes')
with open('../../../keras/deep-learning-vm/notebooks/targetvars.pickle', 'rb') as handle:
    y = pickle.load(handle, encoding='bytes')

print(np.array(X).shape)
print(np.array(y).shape)

(1877, 200, 280, 3)
(1877,)


In [27]:
baseline = y.mean()
print(baseline)

0.503462972829


In [28]:
# Change size of images so it's not so huge when we pass it through keras
import cv2
new_X = []
for row in range(len(X)):
    new_X.append(cv2.resize(X[row], (140,100)))
np.array(new_X).shape

(1877, 100, 140, 3)

In [29]:
# Prepare data for training and testing
X = np.array(new_X)
X = X.astype('float32')
X /= 255.0
y = np.array(y)

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75)

In [31]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras.models import model_from_json

In [32]:
# Model creation by machinelearningmastery
num_classes=2
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(100,140,3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [33]:
# Compile the model
epochs = 50
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
# print(model.summary())

In [34]:
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=32)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 1407 samples, validate on 470 samples
Epoch 1/50
1407/1407 [==============================] - 139s - loss: 0.7484 - acc: 0.5146 - val_loss: 0.6968 - val_acc: 0.4468
Epoch 2/50
1407/1407 [==============================] - 140s - loss: 0.6926 - acc: 0.5281 - val_loss: 0.7031 - val_acc: 0.4468
Epoch 3/50
1407/1407 [==============================] - 143s - loss: 0.6908 - acc: 0.5366 - val_loss: 0.6983 - val_acc: 0.4532
Epoch 4/50
1407/1407 [==============================] - 142s - loss: 0.6830 - acc: 0.5665 - val_loss: 0.7036 - val_acc: 0.4872
Epoch 5/50
1407/1407 [==============================] - 143s - loss: 0.6721 - acc: 0.5842 - val_loss: 0.6942 - val_acc: 0.4979
Epoch 6/50
1407/1407 [==============================] - 141s - loss: 0.6434 - acc: 0.6404 - val_loss: 0.6728 - val_acc: 0.5723
Epoch 7/50
1407/1407 [==============================] - 143s - loss: 0.6194 - acc: 0.6588 - val_loss: 0.6846 - val_acc: 0.5936
Epoch 8/50
1407/1407 [==============================] - 142s - l

In [35]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [36]:
# Try running from json
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [41]:
# evaluate loaded model on test data
loaded_model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
score = loaded_model.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

acc: 70.43%


In [40]:
y_pred = loaded_model.predict_classes(X_test)
y_pred

470/470 [==============================] - 8s     


array([0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0,
       1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 1,

## Conclusion
Neural Networking increased the accuracy of my model of baseline: 50% up to 70%. If I wanted to increase the score, I think the following factors could help increase the accuracy.
- Image transformations
- More epochs
- Alter parameters/layers used
- More data